## Intro to SQL with SQLite: Exploring International Rugby Matches

- Author: Izaan Khudadad
- Email : ikhudada@charlotte.edu
- Affiliation: University of North Carolina at Charlotte

Categories:
- Data Wrangling
- Descriptive Statistics 
- SQL

### Introduction

In the following activity, you will use data compiled by Lyle Begbie from Kaggle, which contains detailed results from international rugby union matches played between 1871 and 2022. The dataset includes match dates, home and away teams, scores, competitions, venues, countries, and flags for World Cup and neutral-site matches.

Each observation represents a single rugby match, providing a rich source of information to explore team performances, scoring patterns, and trends over time.

Using this data, you will practice fundamental SQL skills within Python, while also learning how to perform descriptive data analysis and aggregation on real sports data.

By the end of this activity you should be able to:
1. Use SQLite to write and run basic SQL queries, including `SELECT`, `WHERE`, `GROUP BY`, and `ORDER BY`.
2. Calculate summary statistics such as averages and counts to answer questions about international rugby.
3. Understand how to use SQL for data wrangling and exploration within a Python notebook.
4. Begin to think about how simple data aggregations and filters can reveal meaningful insights from sports data.

## Data

The Dataset contains over 2600 International Rugby Union Results

[International Rugby Results CSV](https://github.com/schuckers/Charlotte_SCORE_Summer25/blob/main/Modules/Intro%20to%20SQL%20with%20Rugby%20by%20Izaan%20Khudadad/results.csv)

| Variable | Description |
|--------------------|-----------------------------------------------------------------------------|
| date              | Date when the match was played (format: YYYY-MM-DD)      |
| home_team              | Name of the team playing at home      |
| away_team             | Name of the team playing away |
| home_score         | Number of points scored by the home team  |
| away_score            | Number of points scored by the away team    |
| competition               | Name of the competition or series (often includes the year)   |
| stadium              | Name of the stadium where the match was played  |
| city             | City where the match took place   |
| country                | Country where the match was hosted |
| neutral           | Boolean indicating if the match was on neutral ground (True/False) |
| world_cup               | Boolean indicating if the match was part of the Rugby World Cup (True/False)  |

**Data Source**

[Kaggle Rugby CSV](https://www.kaggle.com/datasets/lylebegbie/international-rugby-union-results-from-18712022/data)

### Learning Goals
In this module, students will explore how to use SQL to analyze international rugby match data. By writing, executing, and interpreting SQL queries on this dataset, students will practice core data science skills such as:
- Filtering data using `WHERE` clauses to focus on subsets of interest
- Computing summary statistics with `GROUP BY` and aggregate functions (e.g., `COUNT`, `AVG`)
- Sorting and ranking results to identify top teams, competitions, or scoring patterns
- Extracting insights from dates and categorical variables
- Developing familiarity with relational database tools like SQLite for data exploration

## Material

This block of code loads a rugby dataset and sets it up in a way that lets us use SQL commands to explore the data — all inside Python!

In [1]:
import pandas as pd
import sqlite3

# Load the rugby dataset
df = pd.read_csv("https://raw.githubusercontent.com/schuckers/Charlotte_SCORE_Summer25/refs/heads/main/Modules/Intro%20to%20SQL%20with%20Rugby%20by%20Izaan%20Khudadad/results.csv")

# Create an in-memory SQLite database
conn = sqlite3.connect(":memory:")

# Store the dataframe as a SQL table named 'matches'
df.to_sql('matches', conn, index=False, if_exists='replace')

# Preview the first few rows using SQL
pd.read_sql_query("SELECT * FROM matches LIMIT 5;", conn)

,date,home_team,away_team,home_score,away_score,competition,stadium,city,country,neutral,world_cup
0,1871-03-27,Scotland,England,1,0,1871 Scotland v England International,Raeburn Place,Edinburgh,Scotland,0,0
1,1872-02-05,England,Scotland,2,1,1871–72 Home Nations International,The Oval,London,England,0,0
2,1873-03-03,Scotland,England,0,0,1872–73 Home Nations International,West of Scotland F.C.,Glasgow,Scotland,0,0
3,1874-02-23,England,Scotland,1,0,1873–74 Home Nations International,The Oval,London,England,0,0
4,1875-02-15,England,Ireland,2,0,1874–75 Home Nations rugby union matches,The Oval,London,England,0,0


SELECT date, home_team, away_team, home_score, away_score
→ Choose only these columns to display.

FROM matches
→ Look in the matches table we loaded into SQLite.

WHERE home_team = 'New Zealand'
→ Only keep rows where the home team was New Zealand.

LIMIT 5
→ Only show the first 5 matches (to keep the output small)

In [2]:
# Show all matches played by New Zealand as the home team
query = """
SELECT date, home_team, away_team, home_score, away_score
FROM matches
WHERE home_team = 'New Zealand'
LIMIT 5;
"""
pd.read_sql_query(query, conn)


,date,home_team,away_team,home_score,away_score
0,1905-09-02,New Zealand,Australia,14,3
1,1913-09-06,New Zealand,Australia,30,5
2,1913-09-13,New Zealand,Australia,25,13
3,1913-09-20,New Zealand,Australia,5,16
4,1921-08-13,New Zealand,South Africa,13,5


SELECT date, home_team, away_team, home_score + away_score AS total_points
→ Choose to display the match date, teams, and a new calculated column total_points that adds up home_score and away_score.

FROM matches
→ Use the matches table in your SQLite database.

ORDER BY total_points DESC
→ Sort the matches by total_points in descending order, so the highest-scoring games come first.

LIMIT 10
→ Only show the top 10 matches.

In [3]:
# Top 10 highest total scoring matches
query = """
SELECT date, home_team, away_team, home_score + away_score AS total_points
FROM matches
ORDER BY total_points DESC
LIMIT 10;
"""
pd.read_sql_query(query, conn)


,date,home_team,away_team,total_points
0,2023-09-29,New Zealand,Italy,113
1,1998-06-27,South Africa,Wales,109
2,1999-10-14,New Zealand,Italy,104
3,2001-02-17,England,Italy,103
4,1997-06-21,New Zealand,Argentina,101
5,1999-06-19,South Africa,Italy,101
6,2004-06-12,Argentina,Wales,94
7,2010-06-12,New Zealand,Ireland,94
8,1996-06-15,New Zealand,Scotland,93
9,1997-11-08,Italy,South Africa,93


SELECT home_team, AVG(home_score) AS avg_home_score
→ For each team that has played as the home team, calculate the average home score.
The result will show the team name and their average score when playing at home.

FROM matches
→ Use the matches table in the database.

GROUP BY home_team
→ This groups all matches by the home_team so that AVG(home_score) is calculated separately for each team.

ORDER BY avg_home_score DESC
→ Sort the results from the highest to lowest average home score.

LIMIT 10
→ Only display the top 10 teams with the highest average scores at home.

In [4]:
# Average home score by home team
query = """
SELECT home_team, AVG(home_score) AS avg_home_score
FROM matches
GROUP BY home_team
ORDER BY avg_home_score DESC
LIMIT 10;
"""
pd.read_sql_query(query, conn)


,home_team,avg_home_score
0,New Zealand,29.144487
1,South Africa,25.697115
2,Australia,22.604563
3,Argentina,18.914286
4,France,18.720238
5,England,17.730769
6,Wales,16.303279
7,Ireland,14.842254
8,Italy,14.709402
9,Scotland,13.255072


SELECT *
→ Select all columns from the matches table.

FROM matches
→ Use the matches table we loaded into SQLite.

WHERE date >= '2010-01-01'
→ Filter to include only matches played on or after January 1, 2010.

LIMIT 5
→ Show only the first 5 results, to keep the output small and easy to read.

In [5]:
# Matches since 2010
query = """
SELECT *
FROM matches
WHERE date >= '2010-01-01'
LIMIT 5;
"""
pd.read_sql_query(query, conn)


,date,home_team,away_team,home_score,away_score,competition,stadium,city,country,neutral,world_cup
0,2010-02-06,Ireland,Italy,29,11,2010 Six Nations Championship,Croke Park,Dublin,Ireland,0,0
1,2010-02-06,England,Wales,30,17,2010 Six Nations Championship,Twickenham,London,England,0,0
2,2010-02-07,Scotland,France,9,18,2010 Six Nations Championship,Murrayfield,Edinburgh,Scotland,0,0
3,2010-02-13,France,Ireland,33,10,2010 Six Nations Championship,Stade de France,Saint-Denis,France,0,0
4,2010-02-13,Wales,Scotland,31,24,2010 Six Nations Championship,Millennium Stadium,Cardiff,Wales,0,0


### Conclusion
In the provided material, students explored how to use SQL to analyze international rugby match data. By writing, running, and interpreting SQL queries on a real sports dataset, students practiced core data science skills such as:
- Filtering data using `WHERE` clauses to focus on specific subsets
- Computing summary statistics and aggregates with `GROUP BY` and functions like `AVG` and `COUNT`
- Sorting and ranking data using `ORDER BY` to identify patterns and top results
- Using date filters to explore trends over time
- Combining Python and SQL to perform efficient data wrangling inside a notebook


### Activity ###

Using what you’ve learned, write and run SQL queries to answer these questions. Each can be answered by modifying the examples above.

1. **Which countries have hosted the most rugby matches?**  
   (Hint: `GROUP BY country`, `COUNT(*)`)

2. **What is the average total score (home + away) for matches in the World Cup?**  
   (Hint: `WHERE world_cup = 1`, then `AVG(home_score + away_score)`)

3. **Which home team has the highest average winning margin?**  
   (Hint: create `home_score - away_score` as a new column and `AVG()` it)

4. **List all matches played on neutral sites.**

5. **Pick your favorite team:**  
   Write a query to list the last 5 matches they played, whether at home or away.